# In this file we create an ANN model ( testing Purpose ) and fine tune this model for production and deployment.

In [1]:
##  Import Libraries
import keras
from keras.models import Sequential 

keras.__version__



'3.6.0'